In [1]:
import torch

: 

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

In [1]:

def ising_hamiltonian(num_qubits):
    coeffs = [1.0] * (num_qubits - 1)
    obs_list = [qml.PauliZ(i) @ qml.PauliZ(i + 1) for i in range(num_qubits - 1)]
    return qml.Hamiltonian(coeffs, obs_list)

# Define the number of qubits
num_qubits = 5

# Define the quantum device
dev = qml.device("qiskit.aer", wires=num_qubits)

# Define the ansatz circuit
def ansatz(params, wires):
    for i in range(0, len(wires), 2):
        qml.RY(params[i], wires=wires[i])
        if i + 1 < len(wires):
            qml.RY(params[i + 1], wires=wires[i + 1])
            qml.CNOT(wires=[wires[i], wires[i + 1]])

# Define the quantum node
@qml.qnode(dev)
def circuit(params):
    ansatz(params, wires=range(num_qubits))
    return qml.expval(ising_hamiltonian(num_qubits))

# Define the cost function
def cost(params):
    return circuit(params)

# Set up the optimizer
opt = AdamOptimizer(0.1)

# Initialize random parameters
params = np.random.rand(2 * num_qubits - 1)

# Optimize the parameters to find the ground state
iterations = 10000
for i in range(iterations):
    params = opt.step(cost, params)
    if i % 10 == 0:
        energy = circuit(params)
        print(f"Iteration {i+1}/{iterations} - Current energy: {energy}")

# Print the final result
final_energy = circuit(params)
print(f"Optimized parameters: {params}")
print(f"Ground state energy: {final_energy}")

# Find states at different energies
energies_to_check = [-2.0, -1.5, -1.0, -0.5]
for target_energy in energies_to_check:
    params = opt.step(lambda p: (circuit(p) - target_energy)**2, params)
    state = dev._state
    print(f"Target energy: {target_energy}")
    print(f"State at target energy: {state}")

NameError: name 'qml' is not defined

In [3]:
import pennylane as qml
# import torch
# from torch.autograd import Variable
import itertools

In [4]:


print('1')

def generate_graph_hamiltonian(states, edge_energies, grid_size):
    num_vertices = grid_size**2

    hamiltonian_coeffs = []
    hamiltonian_ops = []

    # Add terms for the vertices (state terms)
    for vertex_state in states:
        for vertex_index in range(num_vertices):
            op = qml.PauliZ(vertex_index)
            hamiltonian_ops.append(vertex_state * op)
            hamiltonian_coeffs.append(1.0)

    # Add terms for the edges (edge energy terms)
    for row in range(grid_size):
        for col in range(grid_size):
            vertex_index = row * grid_size + col

            # Right edge
            if col < grid_size - 1:
                right_vertex_index = row * grid_size + (col + 1)
                state_pair = (states[vertex_index], states[right_vertex_index])
                edge_energy = edge_energies.get(state_pair, 0.0)

                op1 = qml.PauliZ(vertex_index)
                op2 = qml.PauliZ(right_vertex_index)

                hamiltonian_ops.append(state_pair[0] * op1 @ state_pair[1] * op2)
                hamiltonian_coeffs.append(edge_energy)

            # Bottom edge
            if row < grid_size - 1:
                bottom_vertex_index = (row + 1) * grid_size + col
                state_pair = (states[vertex_index], states[bottom_vertex_index])
                edge_energy = edge_energies.get(state_pair, 0.0)

                op1 = qml.PauliZ(vertex_index)
                op2 = qml.PauliZ(bottom_vertex_index)

                hamiltonian_ops.append(state_pair[0] * op1 @ state_pair[1] * op2)
                hamiltonian_coeffs.append(edge_energy)

    return qml.Hamiltonian(coeffs=hamiltonian_coeffs, observables=hamiltonian_ops)

print(2)
def extract_vertex_colors(state, states):
    vertex_colors = []

    # Extract the color for each vertex
    for vertex_index, vertex_state in enumerate(states):
        op = qml.PauliZ(vertex_index)
        expectation_value = qml.expval(op)(state)
        color = int(expectation_value.item())
        vertex_colors.append((vertex_index, color))

    return vertex_colors
print(3)
# Define the possible states for each vertex
states = [1, 2, 3]

# Define the energy dictionary for edges
edge_energies = {(1, 1): 0.1, (1, 2): 0.2, (1, 3): 0.3,
                 (2, 1): 0.2, (2, 2): -0.1, (2, 3): 0.4,
                 (3, 1): 0.3, (3, 2): 0.4, (3, 3): -0.2}

# Define the grid size
grid_size = 2

# Generate the Hamiltonian
hamiltonian = generate_graph_hamiltonian(states, edge_energies, grid_size)

# Initialize the optimizer
optimizer = torch.optim.Adam([Variable(torch.randn(num_vertices, requires_grad=True))], lr=0.1)

# Number of optimization steps
steps = 100

print('hello')
# Optimization loop to find ground states
for i in range(steps):
    optimizer.zero_grad()
    params = optimizer.param_groups[0]['params']
    current_energy = qml.expval(hamiltonian)(params)

    # Define a cost function as the energy
    cost = current_energy

    cost.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:
        print(f"Step {i + 1}/{steps} - Energy: {current_energy.item()}")

# Extract the optimized parameters
optimal_params = optimizer.param_groups[0]['params'][0].detach().numpy()

# Create a PennyLane QNode with the optimal parameters
@qml.qnode(dev, interface="torch")
def quantum_circuit(params):
    for i in range(num_vertices):
        qml.RX(params[i], wires=i)
    return qml.expval(qml.PauliZ(0))

# Run the quantum circuit with the optimal parameters
state = quantum_circuit(optimal_params)

# Extract vertex colors
vertex_colors = extract_vertex_colors(state, states)

# Print the results
print("\nOptimal Parameters:", optimal_params)
print("Ground State Energy:", current_energy.item())
print("Vertex Colors:", vertex_colors)


1
2
3


TypeError: unsupported operand type(s) for @: 'Hamiltonian' and 'int'

: 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def plot_colored_graph(vertex_colors, edge_energies, grid_size):
    G = nx.grid_2d_graph(grid_size, grid_size)
    pos = {(x, y): (y, -x) for x, y in G.nodes()}

    # Plot edges with edge energies as labels
    edge_labels = {(i, j): edge_energies.get((vertex_colors[i][1], vertex_colors[j][1]), '')
                   for i, j in G.edges()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')

    # Plot vertices with colors
    node_colors = [vertex_colors[i][1] for i in range(len(vertex_colors))]
    nx.draw(G, pos, with_labels=True, node_color=node_colors, cmap=plt.cm.get_cmap('viridis'))

    plt.show()

# Assuming you have the vertex_colors from the previous code
plot_colored_graph(vertex_colors, edge_energies, grid_size)
